In [30]:
import json
import numpy as np
import h5py
import matplotlib.pyplot as plt
import tqdm
from skimage.feature import ORB
from skimage.color import rgb2gray
import os
import random



# Load data

In [9]:
# map
with open("Data/database/database_lite.json","r") as f:
    m_idx = json.load(f)
    m_imgs = np.array(m_idx["im_paths"])
    m_loc=np.array(m_idx["loc"])

# query
with open("Data/query/query_lite.json","r") as f:
    q_idx=json.load(f)
    q_imgs=np.array(q_idx["im_paths"])
    q_loc=np.array(q_idx["loc"])

# loading the relevance judgements
with h5py.File("Data/london_lite_gt.h5","r") as f:
    fovs = f["fov"][:]
    sim = f["sim"][:].astype(np.uint8)

In [37]:
def get_descriptors(m_imgs, n_keypoints=10):
    
    # Initialize the ORB descriptor
    descriptor_extractor = ORB(n_keypoints=n_keypoints)
    # Initialize the data structure that will contain all the descriptors
    descriptors = None

    for img_name in m_imgs:
        #img = Image.open(os.path.join('data_image_retrieval/', img_name)).convert()
        #img = np.asarray(img)
        img = plt.imread(os.path.join('Data/', img_name))
        img = rgb2gray(img)

        # Extract ORB descriptors
        descriptor_extractor.detect_and_extract(img)

        descriptors_img = descriptor_extractor.descriptors  # descriptors (the feature vectors)

        if descriptors is None:
            descriptors = descriptors_img
        else:
            descriptors = np.vstack( (descriptors, descriptors_img))

    return descriptors.reshape((len(m_imgs), -1))

In [38]:
img2vector = dict(zip(m_imgs, descriptors))

In [42]:
file_path = m_imgs[random.randint(0,len(m_imgs)-1)]

img2vector[file_path].shape

(2560,)

In [33]:
X_raw = []
Y = []

for i in range(fovs.shape[0]): #500
    for j in range(fovs.shape[1]): #1000
        X_raw.append((q_imgs[i],m_imgs[j]))
        Y.append(fovs[i][j])


X_raw = np.array(X_raw)
Y = np.array(Y)

print(X_raw.shape)
print(Y.shape)

(500000, 2)
(500000,)


In [36]:
img2vector

{'database/images/tYvSrmSTLYIQIjYC7jiq2w.jpg': array([[ True,  True,  True, ..., False, False,  True],
        [False,  True,  True, ..., False, False,  True],
        [False,  True, False, ...,  True, False,  True],
        ...,
        [False, False, False, ..., False, False,  True],
        [ True, False,  True, ..., False, False,  True],
        [ True,  True, False, ..., False, False,  True]]),
 'database/images/Zxx10lHeo_e3Db8HijPcOA.jpg': array([[ True,  True, False, ...,  True, False, False],
        [False, False,  True, ...,  True,  True,  True],
        [ True, False,  True, ...,  True, False, False],
        ...,
        [ True,  True, False, ...,  True,  True,  True],
        [False,  True, False, ..., False, False,  True],
        [ True, False, False, ..., False, False,  True]]),
 'database/images/iW-aDnaQ_yrHMi2ClqKojQ.jpg': array([[False, False,  True, ...,  True, False, False],
        [False,  True,  True, ...,  True,  True, False],
        [False,  True,  True, ...,